# Лабораторная работа

Давайте убедимся, что необходимые нам для этой работы библиотеки уже установлены.

In [1]:
!pip install -q numpy tensorflow_datasets tensorflow

In [2]:
# %pip list

# Описание набора данных
В данной работе целью исследования будет набор данных Titanic. Данный набор содержит информацию о различных пассажирах корабля "Титаник", затонувшего в ночь на 15-е апреля 1912 года. Некоторое количество пассажиров спаслось, чему способствовало множество различных факторов, включая их пол, возраст, на какой палубе находилась их кабина, социальный статус, и т.д. Мы предлагаем вам натренировать нейронную сеть для бинарной классификации, способную предсказывать вероятность спасения человека на основе его данных.  

Набор данных состоит из различных признаков, описывающих информацию о пассажирах. Каждая строка таблицы - отдельный пассажир, вся информация о нем содержится в его строке. В столбце survived находится бинарная метка (0 или 1), означающая, спасся ли человек с корабля (1) или нет (0). Вашей задачей является изучить набор данных, корректно определить решаемую задачу, а затем построить модель, которая будет способна предсказывать вероятность спасения и посчитать F1-метрику на тестовом наборе данных.

Для загрузки набора данных мы будем использовать библиотеку tensorflow_datasets. 

Импортируем данную библиотеку, а затем с помощью нее загрузим набор данных "titanic". В процессе загрузки сразу разделим его на тренировочную, валидационную и тестовую выборку. Еще одну часть датасета мы зарезеривировали для оценки вашей модели  системой проверки на образовательной платформе.

In [3]:
import tensorflow_datasets as tfds
ds_train_tf, ds_validation_tf, ds_test_tf, ds_prediction_tf = tfds.load(
    name='titanic',
    split=['train[:70%]', 'train[70%:80%]', 'train[80%:90%]', 'train[90%:]']
)

/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


По умолчанию библиотека загружает данные в формате TensorFlow Dataset. Этот формат позволяет оперировать данными на жестком диске без предварительной загрузки их в память, что позволят обучать модели на данных, превышающих размер оперативной памяти вашего устройства.

Набор данных Titanic является достаточно небольшим набором и мы уверены, что оперативной памяти вашего устройства хватит для хранения его целиком, поэтому это в данный момент не играет решающей роли. Однако, вам будет полезно научиться работать с ним.

Примеры использования и различных функций этого формата данных вы можете найти в документации или Jupyter Notebook'е данного курса, описывающем работу с библиотекой Keras.

Давайте выведем первую строку тренировочного набора данных для ознакомления.

In [4]:
for x in ds_train_tf.take(1):
    print(f"x={x}")

x={'age': <tf.Tensor: shape=(), dtype=float32, numpy=30.0>, 'boat': <tf.Tensor: shape=(), dtype=string, numpy=b'Unknown'>, 'body': <tf.Tensor: shape=(), dtype=int32, numpy=-1>, 'cabin': <tf.Tensor: shape=(), dtype=string, numpy=b'Unknown'>, 'embarked': <tf.Tensor: shape=(), dtype=int64, numpy=2>, 'fare': <tf.Tensor: shape=(), dtype=float32, numpy=13.0>, 'home.dest': <tf.Tensor: shape=(), dtype=string, numpy=b'Sarnia, ON'>, 'name': <tf.Tensor: shape=(), dtype=string, numpy=b'McCrie, Mr. James Matthew'>, 'parch': <tf.Tensor: shape=(), dtype=int32, numpy=0>, 'pclass': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'sex': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'sibsp': <tf.Tensor: shape=(), dtype=int32, numpy=0>, 'survived': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'ticket': <tf.Tensor: shape=(), dtype=string, numpy=b'233478'>}


2022-11-26 12:30:02.202653: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


Как можно заметить, формат текущего датасета представляет собой кортеж из двух элементов - признаков и метки класса. Признаки организованы как словарь с ключами (названиями признаков) и значениями признаков. Данный формат можно использовать напрямую для метода model.fit, однако признаки, несомненно, требуют предобработки.

Итак, к настоящему моменту мы загрузили набор данных, разделили его на тренировочную, валидационную и тестовую выборку и готовы приступить к решению задачи. Вашим текущим заданием является получение максимального значения F1-метрики на тестовом наборе данных, используя все возможности по изучению и визуализации данных, предобработке данных, а также нейронную сеть, выполненную с помощью фреймворка TensorFlow (и его части - Keras).  

В процессе исследования и тренировки запрещено использовать данные из ds_test кроме как для финальной оценки F1 метрики. 

После того, как вы добились максимального на ваш взгляд значения метрики F1 на тестовых данных, выполните предсказание для зарезервированной части датасета (ссылку на данные можно найти на странице с заданием). После чего введите полученные метки в поле ввода на платформе.

Данная лабораторная работа служит не для проверки ваших навыков, а для того, чтобы помочь вам вспомнить основы машинного обучения и работы с соответствующими библиотеками. Мы просим вас попытаться решить задачу самостоятельно, не используя доступную информацию из зарезервированного набора и не пытаясь подогнать модель под идеальную работу с конкретно данным набором, а также просим не использовать внешние источники данных и информацию из соответствующих соревнований на платформе Kaggle.

Вы можете найти для себя полезным пройти снова ноутбук с описанием работы фреймворка Keras и информацией о том, как его использовать. Также по данному фреймворку доступны многочисленные туториалы и примеры на его сайте. Напомним вам, что задача представляет собой бинарную классификацию, поэтому вам необходимо выбрать правильное количество нейронов для последнего слоя и функцию активации.

Успехов!

In [5]:
import pandas as pd

In [6]:
ds_train = tfds.as_dataframe(ds_train_tf)
ds_val = tfds.as_dataframe(ds_validation_tf)
ds_test = tfds.as_dataframe(ds_test_tf)

In [7]:
y_train = ds_train['survived']
y_val = ds_val['survived']
y_test = ds_test['survived']

In [8]:
ds_train = ds_train.drop(['survived'], axis=1)
ds_test = ds_test.drop(['survived'], axis=1)
ds_val = ds_val.drop(['survived'], axis=1)

Увидим, что пропуски заполнены $-1$

In [9]:
ds_train

,age,boat,body,cabin,embarked,fare,home.dest,name,parch,pclass,sex,sibsp,ticket
0,30.0,b'Unknown',-1,b'Unknown',2,13.000000,"b'Sarnia, ON'","b'McCrie, Mr. James Matthew'",0,1,0,0,b'233478'
1,37.0,b'Unknown',98,b'Unknown',2,7.925000,"b'Ruotsinphytaa, Finland New York, NY'","b'Gustafsson, Mr. Anders Vilhelm'",0,2,0,2,b'3101276'
2,28.0,b'9',-1,b'Unknown',2,13.000000,b'Spain',"b'Reynaldo, Ms. Encarnacion'",0,1,1,0,b'230434'
3,18.0,b'Unknown',-1,b'Unknown',2,73.500000,"b'Lyndhurst, England'","b'Davies, Mr. Charles Henry'",0,1,0,0,b'S.O.C. 14879'
4,-1.0,b'Unknown',-1,b'Unknown',0,7.895800,b'Unknown',"b'Gheorgheff, Mr. Stanio'",0,2,0,0,b'349254'
...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,21.0,b'12',-1,b'Unknown',2,10.500000,"b'Guernsey / Wilmington, DE'","b'Rugg, Miss. Emily'",0,1,1,0,b'C.A. 31026'
912,36.0,b'Unknown',-1,b'Unknown',2,7.895800,b'Unknown',"b'Turcin, Mr. Stjepan'",0,2,0,0,b'349247'
913,-1.0,b'Unknown',-1,b'Unknown',2,7.550000,b'Unknown',"b'MacKay, Mr. George William'",0,2,0,0,b'C.A. 42795'
914,39.0,b'2',-1,b'Unknown',2,211.337494,b'Unknown',"b'Kreuchen, Miss. Emilie'",0,0,1,0,b'24160'


In [10]:
ds_train.describe()

,age,body,embarked,fare,parch,pclass,sex,sibsp
count,916.000000,916.000000,916.000000,916.000000,916.000000,916.000000,916.000000,916.000000
mean,23.807041,15.080786,1.485808,34.101990,0.395197,1.292576,0.346070,0.496725
std,18.128681,58.199976,0.822375,54.664940,0.878983,0.841278,0.475976,1.059431
min,-1.000000,-1.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000
25%,5.000000,-1.000000,1.000000,7.895800,0.000000,0.750000,0.000000,0.000000
50%,24.750000,-1.000000,2.000000,14.454200,0.000000,2.000000,0.000000,0.000000
75%,36.000000,-1.000000,2.000000,31.303125,0.000000,2.000000,1.000000,1.000000
max,80.000000,327.000000,3.000000,512.329224,9.000000,2.000000,1.000000,8.000000


In [11]:
ds_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 916 entries, 0 to 915
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        916 non-null    float32
 1   boat       916 non-null    object 
 2   body       916 non-null    int32  
 3   cabin      916 non-null    object 
 4   embarked   916 non-null    int64  
 5   fare       916 non-null    float32
 6   home.dest  916 non-null    object 
 7   name       916 non-null    object 
 8   parch      916 non-null    int32  
 9   pclass     916 non-null    int64  
 10  sex        916 non-null    int64  
 11  sibsp      916 non-null    int32  
 12  ticket     916 non-null    object 
dtypes: float32(2), int32(3), int64(3), object(5)
memory usage: 75.3+ KB


In [12]:
num_features = ['age', 'fare']
# Не берем только body и name
cat_features = ['boat', 'cabin', 'embarked', 'home.dest', 'parch', 'pclass', 'sex',	'sibsp', 'ticket']

Для работы с категориальными признаками воспользуемся библиотекой Target Encoder.

[Ссылка на документацию](https://contrib.scikit-learn.org/category_encoders/targetencoder.html#category_encoders.target_encoder.TargetEncoder)

Используемый нами метод из этой библиотеки опирается на [следующую статью](https://dl.acm.org/doi/pdf/10.1145/507533.507538)


In [13]:
!pip install category_encoders -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
imageio 2.19.3 requires pillow>=8.3.2, but you have pillow 7.2.0 which is incompatible.


In [14]:
import category_encoders as ce

Обучаем Encoder на категориальных признаках тренировочного набора данных и применяем полученное преобразование к соответствующим признакам валидационного и тестового наборов данных.

In [15]:
te = ce.target_encoder.TargetEncoder(cols=cat_features)
ds_train = te.fit_transform(ds_train, y_train)
ds_val = te.transform(ds_val)
ds_test = te.transform(ds_test)

/opt/anaconda3/lib/python3.8/site-packages/category_encoders/target_encoder.py:122: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
/opt/anaconda3/lib/python3.8/site-packages/category_encoders/target_encoder.py:127: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


Объединяем списки в которых хранятся как числовые так и уже закодированные категориальные признаки

In [16]:
features = num_features + cat_features

In [17]:
ds_train[features]

,age,fare,boat,cabin,embarked,home.dest,parch,pclass,sex,sibsp,ticket
0,30.0,13.000000,0.022147,0.288326,0.318612,0.365721,0.319484,0.410526,0.191987,0.329582,0.365721
1,37.0,7.925000,0.022147,0.288326,0.318612,0.098357,0.319484,0.235412,0.191987,0.428571,0.365721
2,28.0,13.000000,0.999999,0.288326,0.318612,0.365721,0.319484,0.410526,0.694006,0.329582,0.365721
3,18.0,73.500000,0.022147,0.288326,0.318612,0.365721,0.319484,0.410526,0.191987,0.329582,0.000904
4,-1.0,7.895800,0.022147,0.288326,0.523316,0.253695,0.319484,0.235412,0.191987,0.329582,0.365721
...,...,...,...,...,...,...,...,...,...,...,...
911,21.0,10.500000,0.999996,0.288326,0.318612,0.365721,0.319484,0.410526,0.694006,0.329582,0.365721
912,36.0,7.895800,0.022147,0.288326,0.318612,0.253695,0.319484,0.235412,0.191987,0.329582,0.365721
913,-1.0,7.550000,0.022147,0.288326,0.318612,0.253695,0.319484,0.235412,0.191987,0.329582,0.365721
914,39.0,211.337494,0.999787,0.288326,0.318612,0.253695,0.319484,0.611354,0.694006,0.329582,0.829416


Произведем стандартизацию для всех предикторов для тренировочного набора данных и применим полученное преобразование для валидационной и тестовой выборок.

In [18]:
from sklearn.preprocessing import StandardScaler


ss = StandardScaler()
ds_train[features] = ss.fit_transform(ds_train[features])
ds_val[features] = ss.transform(ds_val[features])
ds_test[features] = ss.transform(ds_test[features])

Опишем метрики качества модели классификации

In [19]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

Число входов входного слоя нашей НС должно быть равно количеству предикторов, используемых для обучения сети.

In [20]:
input_shape = (len(features),)

Импортируем библиотеки непосредственно для работы с НС

In [21]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras import optimizers
from tensorflow.keras import losses

Создадим архитектуру нашей сети. Будем использовать 4 скрытых слоя. Выходной слой будет нам предсказывать вероятность отнесения к классу 1.

In [22]:
model = keras.Sequential()

model.add(layers.InputLayer(input_shape = input_shape,
                            ))
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(128, activation = 'relu'))
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(32, activation = 'relu'))
model.add(layers.Dense(1, activation = 'sigmoid'))

Воспользуемся модулем остановки обучения в случае, если показатель качества модели перестает существенно изменяться. А также модулем, который позволит нам сохранить в файл "лучшую" версию модели.

In [23]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
es_callback = EarlyStopping(patience=3)
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_f1_m', verbose=1, save_best_only=True, mode='max')

Компилируем модель и производим обучение

In [24]:
model.compile(
    optimizer = optimizers.Adam(learning_rate = 0.01),
    loss = 'BCE',
    metrics = [f1_m, precision_m, recall_m],

)


history = model.fit(ds_train[features],
                    y_train, 
                    epochs=30, batch_size=64, callbacks=[checkpoint, es_callback],
                    validation_data = (ds_val[features], y_val))

Epoch 1/30
10/15 [===================>..........] - ETA: 0s - loss: 0.2431 - f1_m: 0.8999 - precision_m: 0.9038 - recall_m: 0.9080 
Epoch 1: val_f1_m improved from -inf to 0.93309, saving model to model.h5
15/15 [==============================] - 14s 290ms/step - loss: 0.2104 - f1_m: 0.8999 - precision_m: 0.9125 - recall_m: 0.9047 - val_loss: 0.4021 - val_f1_m: 0.9331 - val_precision_m: 0.9710 - val_recall_m: 0.9009
Epoch 2/30
 8/15 [===============>..............] - ETA: 0s - loss: 0.0809 - f1_m: 0.9639 - precision_m: 0.9734 - recall_m: 0.9554
Epoch 2: val_f1_m did not improve from 0.93309
15/15 [==============================] - 0s 28ms/step - loss: 0.0781 - f1_m: 0.9666 - precision_m: 0.9735 - recall_m: 0.9604 - val_loss: 0.2809 - val_f1_m: 0.9331 - val_precision_m: 0.9710 - val_recall_m: 0.9009
Epoch 3/30
 8/15 [===============>..............] - ETA: 0s - loss: 0.0571 - f1_m: 0.9753 - precision_m: 0.9894 - recall_m: 0.9626
Epoch 3: val_f1_m improved from 0.93309 to 0.94078, saving 

Выбираем "лучшую" модель, веса которой мы сохранили в файле

In [25]:
# load 
from keras.models import load_model
model = load_model('model.h5', custom_objects={'f1_m': f1_m, 'precision_m': precision_m, 'recall_m': recall_m})

In [26]:
ds_train[features]

,age,fare,boat,cabin,embarked,home.dest,parch,pclass,sex,sibsp,ticket
0,0.341798,-0.386234,-0.742014,-0.341077,-0.547764,0.071825,-0.473613,0.285015,-0.727472,-0.382689,-0.163563
1,0.728137,-0.479123,-0.742014,-0.341077,-0.547764,-1.303872,-0.473613,-0.828906,-0.727472,0.661350,-0.163563
2,0.231415,-0.386234,1.398089,-0.341077,-0.547764,0.071825,-0.473613,0.285015,1.374624,-0.382689,-0.163563
3,-0.320498,0.721112,-0.742014,-0.341077,-0.547764,0.071825,-0.473613,0.285015,-0.727472,-0.382689,-1.885922
4,-1.369134,-0.479658,-0.742014,-0.341077,1.851442,-0.504596,-0.473613,-0.828906,-0.727472,-0.382689,-0.163563
...,...,...,...,...,...,...,...,...,...,...,...
911,-0.154924,-0.431993,1.398084,-0.341077,-0.547764,0.071825,-0.473613,0.285015,1.374624,-0.382689,-0.163563
912,0.672946,-0.479658,-0.742014,-0.341077,-0.547764,-0.504596,-0.473613,-0.828906,-0.727472,-0.382689,-0.163563
913,-1.369134,-0.485987,-0.742014,-0.341077,-0.547764,-0.504596,-0.473613,-0.828906,-0.727472,-0.382689,-0.163563
914,0.838520,3.243986,1.397627,-0.341077,-0.547764,-0.504596,-0.473613,1.562503,1.374624,-0.382689,2.025621


In [27]:
from sklearn.metrics import f1_score

In [28]:
predict = model.predict(ds_test[features])

f1_score(y_test, predict>=0.5)

5/5 [==============================] - 1s 29ms/step


0.975609756097561

Для нашей тестовой выборки получились очень хорошие результаты. Выполним предсказания для зарезервированной части датасета.

In [29]:
ds_prediction = tfds.as_dataframe(ds_prediction_tf)
ds_prediction = ds_prediction.drop(['survived'], axis=1)

Применим сначала преобразование для категориальных признаков, а затем нормировку уже для всех признаков

In [30]:
ds_prediction = te.transform(ds_prediction)
ds_prediction[features] = ss.transform(ds_prediction[features])

In [31]:
ds_prediction[features]

,age,fare,boat,cabin,embarked,home.dest,parch,pclass,sex,sibsp,ticket
0,-1.267951,-0.093382,1.398093,-0.341077,-0.547764,0.071825,0.859443,0.285015,-0.727472,-0.382689,-0.163563
1,-1.369134,-0.198626,1.398091,-0.341077,-0.240507,-0.504596,-0.473613,-0.828906,1.374624,0.661350,-0.163563
2,-0.982794,1.837610,1.398093,0.236351,1.851442,0.071825,0.859443,1.562503,-0.727472,-0.382689,-0.163563
3,0.286606,-0.450601,-0.742014,-0.341077,-0.547764,-0.504596,-0.473613,-0.828906,-0.727472,-0.382689,-0.163563
4,-1.369134,-0.624177,-0.742014,0.236351,-0.547764,-0.504596,-0.473613,1.562503,-0.727472,-0.382689,-0.163563
...,...,...,...,...,...,...,...,...,...,...,...
126,0.452180,-0.482326,-0.742014,-0.341077,-0.240507,-1.585648,-0.473613,-0.828906,-0.727472,-0.382689,-0.163563
127,-0.265307,-0.481869,-0.742014,-0.341077,-0.547764,0.071825,-0.473613,-0.828906,-0.727472,-0.382689,-0.163563
128,-1.369134,-0.475234,-0.742014,-0.341077,-0.240507,-0.504596,-0.473613,-0.828906,1.374624,-0.382689,-0.163563
129,0.396989,-0.248502,0.009924,-0.341077,-0.547764,0.576932,2.245934,-0.828906,1.374624,1.437733,0.299896


In [32]:
import numpy as np

predict_platform = model.predict(ds_prediction[features])
predictions = np.array([1 if predict_platform[_] >= 0.5 else 0 for _ in range(len(predict_platform))])
predictions

5/5 [==============================] - 0s 5ms/step


array([1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1])